In [1]:
from wxpy import *
bot = Bot()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 测试用的


In [4]:
@bot.register(Group,TEXT)
def auto_accept_friends(msg):
    msg.chat.send(msg.text)
@bot.register(Friend,TEXT)
def auto_accept_friends(msg):
    msg.chat.send(msg.text)

In [3]:
bot.join()